In [1]:
!sudo apt-get update
!sudo apt-get install flex bison

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,669 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,009 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal In

In [282]:
%%writefile lexer.l
%{
#include "parser.tab.h"
%}

%%


[ \t]   ;    /* skip whitespace */
"print"   { return PRINT; }
"true"    { return TRUE; }
"false"   { return FALSE; }
"if"      { return IF; }
"else"    { return ELSE; }
"for"     { return FOR; }
"to"      { return TO; }
"step"    { return STEP; }
"AND"     { return AND; }
"OR"      { return OR; }
"NOT"     { return NOT; }
"<="      { return LE; }
">="      { return GE; }
'<'       { return '<'; }
'>'       { return '>'; }
"=="      {return EQ; }
"!="      {return NEQ; }

[a-zA-Z]+ { yylval = *yytext - 'a'; return ID; }
[0-9]+  { yylval = atoi(yytext); return INTEGER; }
[\n]    { return EOL; }
.       { return yytext[0]; }


%%

int yywrap(void) {
    return 1;
}

Overwriting lexer.l


In [300]:
%%writefile parser.y
%{
#include <stdio.h>
#include <stdlib.h>
extern int yylex(void);
extern char *yytext;
void yyerror(char *);
int variables[26];
%}

%token INTEGER
%token STRING
%token EOL
%token ID
%token PRINT
%token BOOLEAN
%token TRUE FALSE
%left FOR TO STEP
%left IF ELSE
%left AND OR
%nonassoc NOT
%left '+' '-'
%left '*' '/'
%nonassoc '=' '<''>'
%nonassoc LE GE EQ NEQ

%%

program:
    /* empty */
    | program line
    ;

line:
    EOL
    | expr EOL { printf("= %d\n", $1); }
    | for_loop EOL { printf("= %d\n", $1); }
    ;

for_loop:
    FOR ID '=' expr TO expr STEP expr '{' expr '}'
    {
        int i, start = $4, end = $6, step = $8;
        for (i = start; i <= end; i += step) {
            variables[$2] = i;
            yyparse();
        }
        $$ = $10;
    }
    ;

expr:
    INTEGER
    | ID              { $$ = variables[$1];  }
    | ID '=' expr     { variables[$1] = $3; $$ = $3; }
    | '(' expr ')'    { $$ = $2; }
    | expr '+' expr   { $$ = $1 + $3; }
    | expr '-' expr   { $$ = $1 - $3; }
    | expr '*' expr   { $$ = $1 * $3; }
    | expr '/' expr   { $$ = $1 / $3; }

    STRING
    | 'p' '(' expr ')' { printf("%d\n", $3); }      
    | TRUE           { $$ = 1; }
    | FALSE          { $$ = 0; }
    | expr AND expr  { $$ = ($1 && $3); }
    | expr OR expr   { $$ = ($1 || $3); }
    | NOT expr       { $$ = !$2; }
    | expr '<' expr  { $$ = ($1 < $3); }
    | expr '>' expr  { $$ = ($1 > $3); }
    | expr LE expr  { $$ = ($1 <= $3); }
    | expr GE expr  { $$ = ($1 >= $3); }
    | expr EQ expr  { $$ = ($1 == $3); }
    | expr NEQ expr  { $$ = ($1 != $3); }
    | IF '(' expr ')' expr ELSE expr  { $$ = $3 ? $5 : $7; }

%%

int main(void) {
    yyparse();
    return 0;
}

void yyerror(char *s) {
    fprintf(stderr, "error: %s\n", yytext);
}


Overwriting parser.y


In [301]:
!flex lexer.l && bison -d parser.y

In [302]:
!gcc -o parser lex.yy.c parser.tab.c -lfl

In [303]:
%%writefile expresion.txt
a = 2
b = 4
for i = 3 to 7 step 1 { b = b + a }

Overwriting expresion.txt


In [309]:
%%writefile ejemplo.txt
true AND true


Overwriting ejemplo.txt


In [310]:
!./parser <  ejemplo.txt

= 1


In [307]:
%%writefile ejemplo2.txt
a=5
b=4

if (a < b) 5 else 4


Overwriting ejemplo2.txt


In [308]:
!./parser <  ejemplo2.txt

= 5
= 4
= 4


In [305]:
%%writefile ejemplo3.txt

a=4
b=4

a != b

Writing ejemplo3.txt


In [306]:
!./parser <  ejemplo3.txt

= 4
= 4
= 0
